# Работа с базами данных

Материалы:
* Макрушин С.В. Лекция 6: Работа с базами данных
* https://sqliteonline.com/
* https://docs.python.org/3/library/sqlite3.html
* https://www.geeksforgeeks.org/sql-join-set-1-inner-left-right-and-full-joins/
* https://www.datacamp.com/community/tutorials/group-by-having-clause-sql

In [1]:
import sqlite3
import csv
import pandas as pd

## Задачи для совместного разбора

1. Работая с базой данных `Chinook_Sqlite.sqlite`, найдите и выведите на экран имена и фамилии всех заказчиков из Канады

In [2]:
con = sqlite3.connect('data/Chinook_Sqlite.sqlite')
cur = con.cursor()

cur.execute('''
SELECT firstname, lastname 
FROM Customer
WHERE country = ? OR country = ?
''', ('Canada', 'USA'))

# 1. проитерироваться по курсор
# 2. .fetchone()
# 3. .fetchmany(n)
# 4. .fetchall()

print(cur.fetchmany(5))



cur.close()
con.close()

OperationalError: no such table: Customer

2. Найти и вывести на экран названия всех альбомов группы Accept

In [12]:
con = sqlite3.connect('data/Chinook_Sqlite.sqlite')
cur = con.cursor()

cur.execute('''
SELECT title
FROM Artist LEFT JOIN Album
ON Artist.artistid = Album.artistid
WHERE name = 'Accept'
''')

print(cur.fetchall())



cur.close()
con.close()

[('Balls to the Wall',), ('Restless and Wild',)]


3. Создайте базу данных с названием вашей группы. В этой базе данных создайте таблицу Student, содержащую 2 столбца: id и name. Добавьте в таблицу Student информацию о студентах, сидящих с вами по соседству.

In [ ]:
con = sqlite3.connect('data/test.db')
cur = con.cursor()

cur.execute('''
CREATE TABLE Student(id INTEGER, name VARCHAR(50))
''')

cur.execute('''
INSERT INTO Student(id,name) VALUED (1,"Nikita")
''')


#execute

con.commit()
cur.close()
con.close()





## Лабораторная работа 6

1. Создайте файл базы данных c названием `recipes.db`. Создайте объект-курсор. 

In [3]:
con = sqlite3.connect('data/recipes.db')
cur = con.cursor()


In [54]:
cur.execute('''
DROP TABLE Recipe

''')

2. Напишите и выполните SQL-запрос для создания таблицы рецептов `Recipe`. Таблица должна содержать следующие поля:
`id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`. Определитесь с типами и составом ключевых полей.

In [55]:
cur.execute('''
CREATE TABLE Recipe(
id INTEGER PRIMARY KEY, 
name VARCHAR(50), 
minutes INTEGER, 
submitted DATE, 
description VARCHAR(200), 
n_ingredients INTEGER)
''')

con.commit()

3. Напишите и выполните SQL-запрос для создания таблицы отзывов `Review`. Таблица должна содержать следующие поля:
`id`, `user_id`, `recipe_id`, `date`, `rating`, `review`. Определитесь с типами полей, набором ключевых полей. При помощи внешнего ключа соедините две таблицы.

In [56]:
cur.execute('''
CREATE TABLE Review(
id INTEGER PRIMARY KEY, 
user_id INTEGER, 
recipe_id INTEGER,
date DATE, 
rating INTEGER, 
review VARCHAR(300),
FOREIGN KEY(recipe_id) REFERENCES Recipe(id))  
''')

con.commit()

4. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в созданные таблицы

In [94]:
recipe = pd.read_csv('data/recipes_sample_with_tags_ingredients.csv', sep=',')

for i, row in recipe.iterrows():
    data = row.loc[['name', 'minutes', 'submitted', 'description', 'n_ingredients']]
    cur.execute(
        '''
        INSERT INTO
        recipe (name, minutes, submitted, description, n_ingredients)
        VALUES(?, ?, ?, ?, ?)
    ''',data
    )
con.commit()

In [96]:
reviews = pd.read_csv('data/reviews_sample.csv', sep=',')

for i, row in reviews.iterrows():
    data = row.loc[['user_id', 'recipe_id', 'date', 'rating', 'review']]
    cur.execute(
        '''
        INSERT INTO review
         (user_id, recipe_id, date, rating, review)
        VALUES(?, ?, ?, ?, ?)
    ''',
        data,
    )
con.commit()

5. Найдите все рецепты, для выполнения которых нужно ровно 10 ингредиентов. Выведите на экран первые 5 из найденных рецептов.

In [110]:
cur.execute('''
SELECT name
FROM recipe
WHERE n_ingredients = 10
''')
cur.fetchall()

[('blepandekager   danish   apple pancakes',),
 ('kelly s creamy cheddar pea salad',),
 ('middle eastern   twice baked potatoes',),
 ('secret ingredient  bbq meatloaf',),
 ('don t knock it until you try it  beet cake',),
 ('just peachy  cobbler',),
 ('magic  chicken noodle soup',),
 ('missy s  sweet potato casserole',),
 ('second only to my meatloaf  meatballs',),
 ('we re going bananas  bread',),
 ('wolf creek inn  macaroni   cheese',),
 ('different   irish soda bread',),
 ('10 minute tuscan tomato soup with parmesan toasts',),
 ('15 layer russian honey cake',),
 ('3 cheese chicken enchiladas',),
 ('3 hour old fashioned oven pot roast',),
 ('3 bean vegetarian chili  goya beans',),
 ('30 minute chicken and rice dinner by accident',),
 ('30 minute cream of mushroom soup',),
 ('4 hour beef stew',),
 ('5 layer amaretto blueberry vanilla bars',),
 ('5 star spaghetti   meatballs',),
 ('5 spice baby back ribs',),
 ('711 kirkwood s barley casserole',),
 ('a different chicken dish',),
 ('a fun

6. Найдите название рецепта, для выполнения которого требуется больше всего времени.

In [113]:
cur.execute('''
SELECT name
FROM recipe
WHERE minutes = (SELECT MAX(minutes) FROM recipe)

''')
cur.fetchall()

[('blueberry liqueur',), ('strawberry liqueur',)]

7. Запросите у пользователя id рецепта и верните информацию об этом рецепте. Если рецепт отсуствует, выведите соответствующее сообщение.

In [124]:
recipe_id = input("Recipe id: ")

cur.execute('''
SELECT name
FROM recipe
WHERE id = ?

''',(recipe_id,))

result = cur.fetchall()

if result == []:
    print("No such an ID")
else:
    print(result)

Recipe id: 234234
No such an ID


8. Найдите кол-во отзывов с рейтингом 5.

In [126]:
cur.execute('''
SELECT COUNT(id)
FROM review
WHERE rating = 5
''')
cur.fetchall()

[(91361,)]

9. Найдите кол-во уникальных рецептов, не имеющих отзывов с рейтингом, меньше 4. 

In [6]:
cur.execute('''
SELECT COUNT(DISTINCT recipe.name) as Name
FROM review
LEFT JOIN recipe ON recipe.id = review.recipe_id
WHERE recipe_id NOT IN(SELECT recipe_id
FROM review
WHERE rating < 4)
ORDER BY recipe_id
''')

cur.fetchall()

[(1184,)]

10. Найдите кол-во рецептов, опубликованных в 2010 году и имеющих длину не менее 15 минут.

In [14]:
cur.execute('''
SELECT COUNT(id)
FROM recipe
WHERE id NOT IN (SELECT id
FROM recipe
WHERE minutes < 14 OR submitted NOT LIKE '2010%')
''')

cur.fetchall()

[(1322,)]

11. Выберите id рецепта, название рецепта, id пользователя, оставившего отзыв, дату отзыва и рейтинг для тех рецептов, которые имеют не менее 3 ингредиентов. Отсортируйте результат по id рецепта.

In [13]:
cur.execute('''
SELECT recipe.id, recipe.name, review.user_id, review.date, review.rating
FROM recipe
LEFT JOIN review ON recipe.id = review.recipe_id
WHERE recipe.n_ingredients > 2 AND review.rating IS NOT NULL
ORDER BY recipe.id
''')

cur.fetchall()

[(48, 'full cool  macaroni and cheese', 32421, '2002-03-15', 0),
 (48, 'full cool  macaroni and cheese', 68674, '2004-05-03', 2),
 (55, 'little kick  jalapeno burgers', 53959, '2006-01-12', 4),
 (55, 'little kick  jalapeno burgers', 165567, '2006-03-31', 5),
 (55, 'little kick  jalapeno burgers', 851190, '2010-05-23', 5),
 (55, 'little kick  jalapeno burgers', 1060485, '2009-04-07', 5),
 (66, 'second only to my meatloaf  meatballs', 8679, '2003-03-17', 5),
 (66, 'second only to my meatloaf  meatballs', 42938, '2002-10-21', 4),
 (66, 'second only to my meatloaf  meatballs', 124416, '2010-07-08', 5),
 (66, 'second only to my meatloaf  meatballs', 133174, '2010-07-04', 5),
 (66, 'second only to my meatloaf  meatballs', 136004, '2008-06-05', 5),
 (66, 'second only to my meatloaf  meatballs', 136813, '2007-04-14', 5),
 (66, 'second only to my meatloaf  meatballs', 176798, '2010-10-02', 5),
 (66, 'second only to my meatloaf  meatballs', 314792, '2008-02-04', 5),
 (66, 'second only to my meat